# Elliptic curve cryptography 
## Tutorial/playground (part 2)

In [ ]:
import keccak from 'keccak';
import * as Utils from './src/util';
import * as ed25519 from '@noble/ed25519';

> this is about vectors of scalars (ScalarVector) and of points (PointVector).
   we can think of a vector as an array of data.

In [ ]:
const v1 = []
const v2 = []
for (let i = BigInt("1"); i <= BigInt("6"); ++i) {
    let s;
    if (i < BigInt("4")) {
        s = new Utils.Scalar(i);
        v1.push(await s.get_value()); 
    } else {
        s = new Utils.Scalar(i);
        v2.push(await s.get_value());
    }
}

const sv1 = new Utils.ScalarVector(v1);
const sv2 = new Utils.ScalarVector(v2);
await (await sv1.get_value()).forEach(async (s, i) => console.log(`sv1[${i}]: ${await s.get_hex_value()}`));
await (await sv2.get_value()).forEach(async (s, i) => console.log(`sv2[${i}]: ${await s.get_hex_value()}`));

### Scalar Vector Operations Examples

In [ ]:
// Addition
let sv3;
const result = async (): Promise<void> => {
    sv3 = await sv1.add(sv2);
    await (await sv3.get_value()).forEach(async (s, i) => console.log(`sv3[${i}]: ${await s.get_hex_value()}`));
}
result();

In [ ]:
// Subtraction
const result = async (): Promise<void> => {
    const sv4 = await sv1.subtract(sv2);
    await (await sv4.get_value()).forEach(async (s, i) => console.log(`sv4[${i}]: ${await s.get_hex_value()}`));
}
result();

In [ ]:
// Mulitiplication (ScalarVector - ScalarVector)
const result = async (): Promise<void> => {
    const sv5 = await sv1.multiply(sv2);
    await (await sv5.get_value()).forEach(async (s, i) => console.log(`sv5[${i}]: ${await s.get_hex_value()}`));
}
result();

In [ ]:
// Sum of all
const result = async (): Promise<void> => {
    console.log(await (await sv1.sum_of_all()).get_hex_value())
}
result();

In [ ]:
// Negate
const result = async (): Promise<void> => {
    const sv7 = await sv1.negate();
    await (await sv7.get_value()).forEach(async (s, i) => console.log(`sv7[${i}]: ${await s.get_hex_value()}`));
}
result();

In [ ]:
// Dot product x1 ** x2
const result = async (): Promise<void> => {
    const s = await sv1.pow(sv2);
    console.log(`Dot product sv1 ** sv2: ${await s.get_hex_value()}`);
}
result();

### Point Vector Example Operations

In [ ]:
// initialize
const v1 = []
const v2 = []
for (let i = BigInt("1"); i <= BigInt("6"); ++i) {
    if (i < BigInt("4"))
        v1.push(ed25519.Point.BASE.multiply(BigInt(i)).toHex()); 
    else 
        v2.push(ed25519.Point.BASE.multiply(BigInt(i)).toHex());
}

const pv1 = new Utils.PointVector(v1);
const pv2 = new Utils.PointVector(v2);
// point vector
const pa1 = await pv1.get_value();
const pa2 = await pv2.get_value();
pa1.forEach(async (p, i) => console.log(`pv1[${i}]: ${await p.toHex()}`));
pa2.forEach(async (p, i) => console.log(`pv2[${i}]: ${await p.toHex()}`));

In [ ]:
// addtion
const result = async (): Promise<void> => {
    const pv3 = await pv1.add(pv2);
    await (await pv3.get_value()).forEach(async (p, i) => console.log(`pv3[${i}]: ${await p.toHex()}`));
}
result();

In [ ]:
// Multiscalar multiplication ScalarVector**PointVector
const result = async (): Promise<void> => {
    const s = await sv1.pow(sv2);
    const expected = ed25519.Point.BASE.multiply(await s.get_value()).toHex();
    const s2 = await sv1.pow(pv2);
    console.log(`Expected Multiscalar Mult: ${expected}`);
    console.log(`Actual Multiscalar Mult: ${await s2.get_hex_value()}`);
}
result();

### please read util.ts code for better understanding of PointVector.

### Exercise: implement Shamir secret sharing
read more: https://en.wikipedia.org/wiki/Shamir%27s_Secret_Sharing

here's the scenario:
* You know a secret. you call n other people (which we call 'players').
* You must give each player a "partial key" so that exactly m people (with m <= n)
* is required to recover the secret. how would you do that?

### Shamir secret sharing allows you to do this!

### here's the actual secret. the players must not know this (until recovery at least)

In [63]:
const actual_secret = new Utils.Scalar(BigInt("123456789"));

* build the secret polynomial: we want m = 3 players to recover the secret
* hence len(poly) should be 3, or in other words, degree of polynomial should be 2.
* set coeff[0] = actual_secret, and the other coeff must be random scalars.

### note: 
* coeff[0] corresponds to x ** 0 = 1
* coeff[1] corresponds to x ** 1 = x,
* coeff[2] corresponds to x ** 2, etc.

### pre_poly = [None for i in range(3)]

In [ ]:
// code here

#### polynomial evaluation poly(x)
    python example:
```python
def poly_eval(x: Scalar, coeff: ScalarVector) -> Scalar:
    powers_x = ScalarVector()
    powers_x.append(Scalar(1))
    for i in range(len(coeff) - 1):
        powers_x.append(x * powers_x[i])
    return powers_x ** coeff
```

### list of n = 5 'players'/x-coord of share coords
##### note: Scalar(0) is not allowed in player_list because poly(0) = secret (which leaks the secret)
```python
player_list = [Scalar(1), Scalar(2), Scalar(3), Scalar(4), Scalar(5)]
```

### build all share coords. these are the "partial keys" to be sent to the players
```python
all_coords = []
for xi in player_list:
    all_coords.append((xi, poly_eval(xi, poly)))
```

### recover the secret: implement the formula in the following link:
https://en.wikipedia.org/wiki/Shamir%27s_Secret_Sharing#Computationally_efficient_approach
#### * coords: set of coords for recovery
#### note: our "division" is (x1 * x2.invert()), NOT (x1 // x2) !!!

In [ ]:
// code here

### here's a 3 coords to recover secret:

#### Experiment: what would happen if 

1. you change some indexes?
2. add/remove coords?

```python
recover_coords = [all_coords[0], all_coords[2], all_coords[4]]

recovered_secret = recover(recover_coords)
if recovered_secret == actual_secret:
    print("\nThe implementation of Shamir secret sharing works!")
else:
    print("\nThere's a problem in implementation of Shamir secret sharing.")
```